# <font class='ign_color'>ROS EN UNA SOLA SEMANA</font>

# Capítulo 4: ROS Actions

<table style="width:100%">
  <tr>
    <th>Tiempo estimado para completarlo: 2'5 horas<br><br>Qué vas a aprender en esta unidad?
<ul>
  <li>Cómo crear un action server</li>
  <li>Cómo crear tu propio mensaje de actions</li>
</ul></th>
    <th><img src="img/drone.jpg" width="291" height="248" /></th> 
  </tr>
</table>

## Parte 2

En tu anterior lección, has aprendido cómo <b>llamar</b> a un action server. En esta lección vas a aprender cómo <b>crear</b> tu propio action server.

<figure>
  <img id="fig-4.5" src="img/action_interface.png"/>
   <center> <figcaption>Fig.4.5 - Action Interface Diagram Copy 2</figcaption></center>
</figure>

## Escribiendo unn action server

<table style="float:left;">
<tr>
<th>
<p style="background:#EE9023;color:white;">Ejercicio 4.11: Testea el Fibonacci Action Server a través del Notebook</p><br>
Ejecuta el siguiente código Python clicando en él y después clicando en el botón de Play en la esquina superior derecha del IPython notebook.<br> 
<img src="img/font-awesome_step-forward.png" style="float:left"/><br>
<br><br>
También puedes pulsar <i>[CTRL]+[Enter]</i> para ejecutarlo.<br>
<br>

Cuando el programa haya terminado, no te olvides de <span style="color:red">hacer restart del Kernel</span>. Esto limpiará todos los nodos generados por ROS a través de los programas Python. Esto es necesario porque los programas Python sólo pueden lanzar <b>un</b> nodo rospy. Debido a que este notebook es simplemente un script de python dividido, te dará una rospy Exception si tratas de ejecutar 2 partes del código de manera consecutiva sin reiniciar el Kernel.
Puedes hacer esto presionando este icono.<br>
<img src="img/refresh_icon.png" style="float:left"/>
<br><br><br>


Lo que hay a continuación es un ejemplo del código de un action server. Cuando lo llamas, el action server generará una secuencia Fibonacci de un orden dado. El mensaje goal del action server debe indicar el orden de la secuencia a calcular; el feedback es la propia secuencia mientras está siendo calculada, y el result es la secuencia Fibonacci final.
<br>
</th>
</tr>
</table>

<p style="background:#3B8F10;color:white;" id="prg-4.11a">Programa Python {4.11a}: fibonacci_action_server.py </p><br>

In [1]:
import rospy

import actionlib

from actionlib_tutorials.msg import FibonacciFeedback, FibonacciResult, FibonacciAction

class FibonacciClass(object):
    
  # create messages that are used to publish feedback/result
  _feedback = FibonacciFeedback()
  _result   = FibonacciResult()

  def __init__(self):
    # crea el action server
    self._as = actionlib.SimpleActionServer("fibonacci_as", FibonacciAction, self.goal_callback, False)
    self._as.start()
    
  def goal_callback(self, goal):
    # este callback es llamado cuando alguien llama al action server.
    # esta es la función que calcula la secuencia Fibonacci
    # y retorna la secuencia al nodo que haya llamado al action server
    
    # variables de ayuda
    r = rospy.Rate(1)
    success = True
    
    
    self._feedback.sequence = []
    self._feedback.sequence.append(0)
    self._feedback.sequence.append(1)
    
    # publica información en la consola para el usuario
    rospy.loginfo('"fibonacci_as": Executing, creating fibonacci sequence of order %i with seeds %i, %i' % ( goal.order, self._feedback.sequence[0], self._feedback.sequence[1]))
    
    # empieza a calcular la secuencia Fibonacci
    fibonacciOrder = goal.order
    for i in xrange(1, fibonacciOrder):
    
      # comprueba que el preempt (cancelación) no ha sido solicitado por el action client
      if self._as.is_preempt_requested():
        rospy.loginfo('The goal has been cancelled/preempted')
        # la siguiente línea, pone el client en estado preempted (goal cancelado)
        self._as.set_preempted()
        success = False
        # finalizamos el cálculo de la secuencia Fibonacci
        break
      
      # construye el siguiente mensaje feedback para ser enviado
      self._feedback.sequence.append(self._feedback.sequence[i] + self._feedback.sequence[i-1])
      # publica el feedback
      self._as.publish_feedback(self._feedback)
      # la secuencia es calcula a 1 Hz
      r.sleep()
    
    # en este punto, o bien el goal se ha completado exitosamente (success==true)
    # o el client ha hecho preempt del goal (success==false)
    # si es success, publicamos el result final
    # si no es success, no publicamos nada en el result
    if success:
      self._result.sequence = self._feedback.sequence
      rospy.loginfo('Succeeded calculating the Fibonacci of order %i' % fibonacciOrder )
      self._as.set_succeeded(self._result)
      
if __name__ == '__main__':
  rospy.init_node('fibonacci')
  FibonacciClass()
  rospy.spin()

<p style="background:#3B8F10;color:white;">Explicación Código del Programa Python: {4.11a}</p><br>
En este caso, el action server está usando un mensaje definido en <i>Fibonacci.action</i>. Este mensaje ha sido creado por ROS en el paquete <i>actionlib_tutorials</i>.
<p style="background:#3B8F10;color:white;">Fin Explicación del Código del Programa Python: {4.11a}</p><br>

<table style="float:left;">
<tr>
<th>
<p style="background:#EE9023;color:white;">Ejercicio 4.12a: Comprueba la estructura del mensaje Fibonacci</p><br>
Comprueba la estructura del mensaje Fibonacci.action, yendo al directorio <i>action</i> del paquete <i>actionlib_tutorials</i>.
</th>
</tr>
</table>

<table style="float:left;">
<tr>
<th>
<p style="background:#EE9023;color:white;">Ejercicio 4.12b: Mira la salida de los tópicos feedback y result de Fibonacci</p><br>

Lanza de nuevo el código python anterior <a href="#prg-4.11a">{4.11a}</a> para tener el action server fibonacci corriendo.<br>
Después ejecuta los siguientes comandos en sus respectivos WebShells.<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p style="background: #FFFFFF">Ejecuta en el WebShell #1: Echo del Result</p><br>
rostopic echo /fibonacci_as/result<br>
</th>
</tr>
</table>
<br><br><br><br><br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p style="background: #FFFFFF">Ejecuta en el WebShell #2: Echo del feedback</p><br>
rostopic echo /fibonacci_as/feedback<br>
</th>
</tr>
</table>
<br><br><br><br><br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p style="background: #FFFFFF">Ejecuta en el WebShell #3: Envia el goal a tu fibonacci server</p><br>
rostopic pub /fibonacci_as/goal actionlib_tutorials/FibonacciActionGoal [TAB][TAB]<br>
</th>
</tr>
</table>
<br><br><br><br><br>

<p style="background:#3B8F10;color:white;">Información para el Ejercicio 4.12b</p><br>
<ul>
<li>
Los mensajes en el código Python se llaman FibonacciGoal, FibonacciResult y FibonacciFeedback, mientras que en los tópicos, se llaman FibonacciActionGoal, FibonacciActionResult y FibonacciActionFeedback.
</li>
<li>
Cual es la diferencia entre <b>FibonacciGoal</b> y <b>FibonacciActionGoal</b>?
</li>
<li>
Básicamente es que el <b>FibonacciGoal</b> está incomplete, no tiene algunos elementos necesarios para ser usado directamente en tópicos. Sólo es usado por objetos Python. <b>FibonacciActionGoal</b> es el que usan los mensajes, y el que debes usar cuando trates con los tópicos directamente.
</li>
</ul>




<p style="background:#AE0202;color:white;">Resultado Esperado para el Ejercicio 4.12b</p><br>
Tras haber llamado a la action, el tópico feedback debería estar publicando el feedback, y el result cuando terminen los cálculos.


<br>
</th>
</tr>
</table>

<table style="float:left;">
<tr>
<th>
<p style="background:#EE9023;color:white;" id="ex-4-13">Ejercicio 4.13: Crea un paquete con un Action Server que mueva el Ardrone en un cuadrado</p><br>
Crea un paquete con un action server que mueva al drone en forma de cuadrado cuando sea llamado.<br>
Llama al action server a través de tópicos y observa el result y el feedback.<br>
Basa tu código en el ejemplo anterior <a href="#prg-4.11a">{4.11a}</a> y en el client que hiciste en el ejercicio 4.6 que movía al drone mientras tomaba fotos.<br>

<p style="background:#3B8F10;color:white;">Información para el Ejercicio 4.13</p>
<ul>
<li>El tamaño de cada lado del cuadrado debe ser especificado en el mensaje del goal como un entero.</li>
<li>El feedback debe publicar el segundo en el que el robot se encuentra mientras realiza el cuadrado.</li>
<li>El result debe publicar el número total de segundos que el drone ha tardado en hacer el cuadrado</li>
<li>Usa el mensaje <span style="color: orange"><i>Test.action</i></span> para este action server. Usa el comando <span style="color: orange"><i>locate Test.action</i></span> para encontrar donde se encuentra definido el mensaje.</li>
</ul>

<p style="background:#AE0202;color:white;">Resultado Esperado para el Ejercicio 4.13</p><br>
El resultado se muestra en la animación de debajo <a href="#fig-4.6">{Fig:4.6}</a>

</th>
</tr>
</table>




<figure>
  <img id="fig-4.6" src="img/drone.gif">
   <center> <figcaption>Fig.4.6 - Ardrone movido a través de comandos enviado por un action server Ex 4.13</figcaption></center>
</figure>

## Cómo crear tu propio mensaje para el action server

<b>Siempre se recomienda usar mensajes de actions ya proporcionados por ROS.</b><br>
Estos puedes ser encontrados en los siguientes paquetes ROS:
<ul>
<li>actionlib</li>
    <ul>
    <li>Test.action</li>
    <li>TestRequest.action </li>
    <li>TwoInts.action</li>
    </ul>
<li>actionlib_tutorials</li>
    <ul>
    <li>Fibonacci.action</li>
    <li>Averaging.action</li>
    </ul>
</ul>
<br>

Sin embargo, en el caso de que necesites crear tu propio mensaje, haz el siguiente ejercicio para saber cómo.

<table style="float:left;">
<tr>
<th>
<p style="background:#EE9023;color:white;">Ejercicio 4.14: Crea tu propio mensaje de action</p><br>
<ol>
  <li>Crea un directorio <i>action</i> dentro de tu paquete</li>
  <li>Crea un fichero NAME.action
      <ul>
        <li>El NAME del fichero del mensaje determinará más tarde el nombre de las clases que se usarán en el <i>action server</i> y/o el <i>action client</i></li>
        <li>
        Recuerda que el fichero NAME.action tiene que contener 3 partes, cada parte separada por 3 guiones.  
        #<span style="color:orange">goal</span><br>
        <span style="color:green">paquete_donde_está_el_mensaje/tipo_de_mensaje nombre_variable_goal</span><br>
        <span style="color:red">---</span><br>
        #<span style="color:orange">result</span><br>
        <span style="color:green">paquete_donde_está_el_mensaje/tipo_de_mensaje nombre_variable_result</span><br>
        <span style="color:red">---</span><br>
        #<span style="color:orange">feedback</span><br>
        <span style="color:green">paquete_donde_está_el_mensaje/tipo_de_mensaje nombre_variable_feedback</span>
        <br><br>
        </li>
        <li>Si no necesitas alguna parte del mensaje (por ejemplo no necesitas mandar un feedback) puedes dejar esa parte vacía, pero debes especificar siempre los guines de separación</li>
      </ul>
  </li>
  <li>Modifica los ficheros CMakeLists.txt y package.xml para incluir la compilación de mensajes de action. Mira la descripción <a href="#custom_compilation">Cómo prepara el CMakeLists.txt y el package.xml para la compilación de mensajes de action</a>
  </li>
</ol>

<br><br>
Cuando todo esté correctamente configurado, simplemente tienes que compilar:<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p style="background: #FFFFFF">Ejecuta en el WebShell #1</p><br>
roscd; cd ..<br>
catkin_make<br>
source devel/setup.bash<br>
rosmsg list | grep NAME<br>
</th>
</tr>
</table>
<br><br><br><br><br><br><br><br>


<table style="float:left;background: #407EAF">
<tr>
<th>
<p style="background: #FFFFFF">Salida del WebShell #1</p><br>
my_custom_action_msg_pkg/NAMEAction<br>
my_custom_action_msg_pkg/NAMEActionFeedback<br>
my_custom_action_msg_pkg/NAMEActionGoal<br>
my_custom_action_msg_pkg/NAMEActionResult<br>
my_custom_action_msg_pkg/NAMEFeedback<br>
my_custom_action_msg_pkg/NAMEGoal<br>
my_custom_action_msg_pkg/NAMEResult<br>
</th>
</tr>
</table>
<br><br><br><br><br><br><br><br><br><br><br>





</th>
</tr>
</table>

<div id="custom_compilation"></div>

## Cómo preparar el CMakeLists.txt y el package.xml para la compilación de mensajes de action

Debes editar 2 ficheros del paquete, como se explicó en Tópicos y Servicios:
<ul>
<li>CMakeLists.txt</li>
<li>package.xml</li>
</ul>

### En el package.xml:

<ol>
<li>
Añade todos los paquetes necesarios para compilar los mensajes.<br>
Si, por ejemplo, una de las variables en tu fichero .action usa un mensaje definido fuera del paquete std_msgs, digamos nav_msgs/Odometry, necesitas importarlo. Para ello, deberás añadir como <b>build_depend</b> el paquete <b>nav_msgs</b> de esta manera:<br>
<span style="color:green">&lt;build_depend&gt;nav_msgs&lt;/build_depend&gt;</span>
</li>
<li>
Por el contrario, si necesitas un paquete para la ejecución de los programas de tu paquete, deberás importar esos paquete como <b>run_depend</b> de esta manera:<br>
<span style="color:green">&lt;run_depend&gt;nav_msgs&lt;/run_depend&gt;</span>
</li>
</ol>

Cuando compilas mensajes de action, es <span style="color: red">siempre</span> necesario añadir el paquete <span style="color: red">actionlib_msgs</span> como build_dependency.<br>
<span style="color:red">&lt;build_depend&gt;actionlib_msgs&lt;/build_depend&gt;</span><br>
Esto es porque estos son los mensajes usados para la generación de los mensajes básicos de action como <i>goal, feedback y result</i>.<br>

Cuando usas python, es <span style="color: red">siempre</span> necesario añadir <span style="color: red">rospy</span> como run_dependency.<br>
<span style="color:red">&lt;run_depend&gt;rospy&lt;/run_depend&gt;</span><br>
Esto es debido a que el módulo python rospy es necesario para ejecutar tus códigos python en ROS.

### En el CMakeLists.txt

Deberás editar 4 funciones dentro del CMakeLists.txt:
<ul>
<li><span style="color:green"><a href="#find_package">find_package()</a></span></li>
<li><span style="color:green"><a href="#add_action_files">add_action_files()</a></span></li>
<li><span style="color:green"><a href="#generate_messages">generate_messages()</a></span></li>
<li><span style="color:green"><a href="#catkin_package">catkin_package()</a></span></li>
</ul>


<span style="color: green" id="find_package">find_package()</span>

Aquí van todos los paquetes necesarios para COMPILAR los mensajes de tópicos, servicios y actions. Simplemente está cogiendo sus paths, no los está importando para usarlos en la compilación. Simplemente es para que futuras funciones del CMakeLists.txt sean capaces de encontrar esos paquetes.
En el package.xml debes colocarlos como <b>build_depend</b>.

<span style="color: green" id="add_action_files">add_action_files()</span>

En esta función estarán todos los mensajes de actions ( que están en el directorio action ) que quieras compilar.<br>
Puedes indicar que coja todos los mensajes dentro del directorio action: DIRECTORY action<br>
o sólo los mensajes que indiques específicamente: FILES my_custom_action.action<br>
En tu caso puedes hacer cualquiera de los 2, como prefieras.

<span style="color: green" id="generate_messages">generate_messages()</span>

Aquí es donde los paquetes necesarios para la compilación de los mensajes son importados.

<span style="color: green" id="catkin_package">catkin_package()</span>

Coloca aquí todos los paquetes que serán necesarios para alguien que ejecute algo de tu paquete.
Todos los paquetes que estén aquí deben estar en el package.xml como <b>run_depend</b>.

Deberías terminar con un CMakeLists.txt similar a este:

Y un package.xml similar a este:

<p style="background:green;color:white;">Nota</p><br>
Date cuenta que no has importado el paquete <b>std_msgs</b> en ninguna parte. Pero sin embargo puedes usar los mensajes declarados ahí en tus fichero .action. Esto es porque este paquete forma parte de los ficheros de sistema del roscore, así que ya están incluidos en los protocolos de compilación, y no necesitas declararlos.

<table style="float:left;">
<tr>
<th>
<p style="background:#EE9023;color:white;">Ejercicio 4.12: Crea un paquete con un action server con un mensaje personalizado para mover el ardrone</p><br>
<ul>
  <li>En el anterior paquete que creaste <a href="#ex-4-13">[Ex 4.13]</a>, crea un nuevo action server para el quadcopter</li>
  <li>El action server recibirá como goal 2 palabras: ARRIBA o ABAJO</li>
  <li>Cuando el action server reciba la palabara ARRIBA, moverá al drone 1 metro hacia arriba</li>
  <li>Cuando el action server reciba la palabara ABAJO, moverá al drone 1 metro hacia abajo</li>
  <li>Como feedback, publicará una vez por segundo qué acción está haciendo (subiendo o bajando)</li>
  <li>Cuando finalice la action, eñ result no devolverá nada</li>
</ul>

<p style="background:#3B8F10;color:white;">Información</p><br>
<ul>
<li>Necesitas crear un nuevo mensaje de action con los valores especificados como <i>String</i>. Este tipo puede ser importado del paquete <i>std_msgs</i>.</li>
<li>La parte del result del mensaje estará vacía.</li>
<li>Ya que estamos hablando de un drone, puedes especificar velocidades en 3 ejes. Deberás hacer eso en orden para mover el drone arriba y abajo.</li>
</ul>
</th>
</tr>
</table>

## Material adicional para aprender más

ROS Actions: http://wiki.ros.org/actionlib<br>
Cómo funcionan las actions de ROS (descripción detallada): http://wiki.ros.org/actionlib/DetailedDescription